<a href="https://colab.research.google.com/github/Vashuki2004/TrendTicker/blob/main/TrendTicker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Trend Ticker.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1PDPxPSbK2lg_mpuG9rUpZ_Otonor4Wuy
"""

!pip install yfinance pandas

import yfinance as yf
import pandas as pd

# Function to fetch stock data
def fetch_stock_data(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1y")  # Fetch data for the last year
    return hist

# Function to calculate the 200-day EMA
def calculate_ema(df, span=200):
    df['200-EMA'] = df['Close'].ewm(span=span, adjust=False).mean()
    return df

# Function to calculate the current price deviation from the 200-EMA
def calculate_current_deviation(df):
    current_close = df['Close'].iloc[-1]
    current_ema = df['200-EMA'].iloc[-1]
    current_deviation = ((current_close - current_ema) / current_ema) * 100
    return current_deviation, current_close, current_ema

# Function to calculate ROE, Debt-to-Equity ratio, and other financial metrics
def calculate_financial_ratios(ticker):
    stock = yf.Ticker(ticker)
    balance_sheet = stock.balance_sheet
    financials = stock.financials

    # Attempt to get Shareholders' Equity from the balance sheet
    equity_keys = ['Stockholders Equity', 'Common Stock Equity']
    for key in equity_keys:
        if key in balance_sheet.index:
            shareholders_equity = balance_sheet.loc[key].iloc[0]
            break
    else:
        raise KeyError("Shareholders' equity not found in balance sheet.")

    # Attempt to get Total Debt from the balance sheet
    total_debt_keys = ['Total Debt', 'Total Liabilities Net Minority Interest']
    for key in total_debt_keys:
        if key in balance_sheet.index:
            total_debt = balance_sheet.loc[key].iloc[0]
            break
    else:
        raise KeyError("Total Debt not found in balance sheet.")

    # Extracting the latest net income
    net_income = financials.loc['Net Income'].iloc[0]

    # Calculating ROE
    roe = (net_income / shareholders_equity) * 100

    # Calculating Debt-to-Equity Ratio
    debt_to_equity_ratio = total_debt / shareholders_equity

    return roe, debt_to_equity_ratio

# Function to calculate EPS and PE Ratio
def calculate_eps_pe(ticker):
    stock = yf.Ticker(ticker)
    # Earnings per share (EPS)
    eps = stock.info.get('trailingEps')
    if eps is None:
        raise KeyError("Earnings per Share (EPS) data not found.")

    # Current price
    current_price = stock.history(period="1d")['Close'].iloc[-1]

    # Calculate PE Ratio
    pe_ratio = current_price / eps
    return eps, pe_ratio

# Function to classify PE ratio based on sector-specific thresholds
def classify_pe_ratio(pe_ratio, sector):
    if pe_ratio < 0:
        return "Avoid"

    sector_pe_thresholds = {
        "Technology": (10, 20, 30, 40),
        "Healthcare": (8, 16, 24, 32),
        "Financial Services": (5, 10, 15, 20),
        "Consumer Cyclical": (10, 18, 25, 32),
        "Consumer Defensive": (10, 18, 22, 26),
        "Energy": (5, 10, 15, 20),
        "Utilities": (8, 14, 20, 25),
        "Basic Materials": (5, 10, 15, 20),
        "Industrials": (8, 14, 20, 26),
        "Real Estate": (10, 16, 22, 28)
        # Add other sectors as needed
    }

    thresholds = sector_pe_thresholds.get(sector, (10, 20, 30, 40))  # Default thresholds

    if pe_ratio < thresholds[0]:
        return "Very Low"
    elif thresholds[0] <= pe_ratio < thresholds[1]:
        return "Low"
    elif thresholds[1] <= pe_ratio < thresholds[2]:
        return "Medium"
    elif thresholds[2] <= pe_ratio < thresholds[3]:
        return "High"
    else:
        return "Very High"

# Function to classify Debt-to-Equity ratio based on sector-specific thresholds
def classify_de_ratio(de_ratio, sector):
    sector_de_thresholds = {
        "Technology": (0.1, 0.5, 1.0, 2.0),
        "Healthcare": (0.2, 0.6, 1.2, 2.5),
        "Financial Services": (1.0, 2.0, 3.0, 5.0),
        "Consumer Cyclical": (0.3, 0.7, 1.5, 3.0),
        "Consumer Defensive": (0.2, 0.6, 1.2, 2.0),
        "Energy": (0.3, 0.8, 1.5, 3.0),
        "Utilities": (0.5, 1.0, 2.0, 3.5),
        "Basic Materials": (0.2, 0.7, 1.5, 2.5),
        "Industrials": (0.3, 0.8, 1.5, 2.5),
        "Real Estate": (0.5, 1.0, 2.0, 3.0)
        # Add other sectors as needed
    }

    thresholds = sector_de_thresholds.get(sector, (0.5, 1.0, 2.0, 3.0))  # Default thresholds

    if de_ratio < thresholds[0]:
        return "Very Low"
    elif thresholds[0] <= de_ratio < thresholds[1]:
        return "Low"
    elif thresholds[1] <= de_ratio < thresholds[2]:
        return "Medium"
    elif thresholds[2] <= de_ratio < thresholds[3]:
        return "High"
    else:
        return "Very High"

# Function to classify EPS based on sector-specific thresholds
def classify_eps(eps, sector):
    sector_eps_thresholds = {
        "Technology": (1, 3, 5, 10),
        "Healthcare": (0.5, 2, 4, 8),
        "Financial Services": (0.5, 2, 4, 8),
        "Consumer Cyclical": (1, 3, 5, 10),
        "Consumer Defensive": (0.5, 2, 4, 8),
        "Energy": (0.5, 2, 4, 8),
        "Utilities": (0.5, 1, 2, 4),
        "Basic Materials": (0.5, 2, 4, 8),
        "Industrials": (0.5, 2, 4, 8),
        "Real Estate": (0.5, 2, 4, 8)
        # Add other sectors as needed
    }

    thresholds = sector_eps_thresholds.get(sector, (1, 3, 5, 10))  # Default thresholds

    if eps < thresholds[0]:
        return "Very Low"
    elif thresholds[0] <= eps < thresholds[1]:
        return "Low"
    elif thresholds[1] <= eps < thresholds[2]:
        return "Medium"
    elif thresholds[2] <= eps < thresholds[3]:
        return "High"
    else:
        return "Very High"

# Function to get the sector and market capitalization of the stock
def get_sector_and_market_cap(ticker):
    stock = yf.Ticker(ticker)
    sector = stock.info.get('sector', 'Sector not available')
    market_cap = stock.info.get('marketCap', 'Market Cap not available')
    return sector, market_cap

# Function to format large numbers in billions, millions, or trillions
def format_large_number(value):
    if value >= 1_000_000_000_000:
        return f"${value / 1_000_000_000_000:.2f} Trillion"
    elif value >= 1_000_000_000:
        return f"${value / 1_000_000_000:.2f} Billion"
    elif value >= 1_000_000:
        return f"${value / 1_000_000:.2f} Million"
    else:
        return f"${value:.2f}"

# Function to get sector-specific discount rates
def get_discount_rate_by_sector(sector):
    sector_discount_rates = {
        "Technology": 0.12,
        "Healthcare": 0.10,
        "Financial Services": 0.08,
        "Consumer Cyclical": 0.09,
        "Consumer Defensive": 0.07,
        "Energy": 0.13,
        "Utilities": 0.06,
        "Basic Materials": 0.11,
        "Industrials": 0.09,
        "Real Estate": 0.08
        # Add other sectors as needed
    }
    return sector_discount_rates.get(sector, 0.10)  # Default rate if sector is not listed

# Function to classify DCF valuation based on sector
def classify_dcf_value(dcf_value, market_cap, sector):
    sector_dcf_thresholds = {
        "Technology": (0.5, 1, 2, 3),
        "Healthcare": (0.5, 1, 2, 3),
        "Financial Services": (0.5, 1, 2, 3),
        "Consumer Cyclical": (0.5, 1, 2, 3),
        "Consumer Defensive": (0.5, 1, 2, 3),
        "Energy": (0.5, 1, 2, 3),
        "Utilities": (0.5, 1, 2, 3),
        "Basic Materials": (0.5, 1, 2, 3),
        "Industrials": (0.5, 1, 2, 3),
        "Real Estate": (0.5, 1, 2, 3)
        # Add other sectors as needed
    }

    thresholds = sector_dcf_thresholds.get(sector, (0.5, 1, 2, 3))  # Default thresholds

    market_cap = market_cap or 0  # Ensure market_cap is not None

    ratio = dcf_value / market_cap if market_cap > 0 else 0

    if ratio < thresholds[0]:
        return "Very Low"
    elif thresholds[0] <= ratio < thresholds[1]:
        return "Low"
    elif thresholds[1] <= ratio < thresholds[2]:
        return "Medium"
    elif thresholds[2] <= ratio < thresholds[3]:
        return "High"
    else:
        return "Very High"

# Function to perform DCF analysis
def perform_dcf_analysis(ticker, sector, growth_rate=0.06, projection_period=5):
    stock = yf.Ticker(ticker)

    # Estimate Free Cash Flow (FCF) for the last year
    fcf = stock.cashflow.loc['Free Cash Flow'].iloc[0]

    # Get the sector-specific discount rate
    discount_rate = get_discount_rate_by_sector(sector)

    # Calculate the present value of projected FCF
    present_value_of_fcfs = 0
    for year in range(1, projection_period + 1):
        fcf_future = fcf * ((1 + growth_rate) ** year)
        present_value_of_fcfs += fcf_future / ((1 + discount_rate) ** year)

    # Calculate the terminal value
    terminal_value = (fcf * ((1 + growth_rate) ** projection_period)) / (discount_rate - growth_rate)
    present_value_of_terminal_value = terminal_value / ((1 + discount_rate) ** projection_period)

    # Total present value
    total_present_value = present_value_of_fcfs + present_value_of_terminal_value

    # Market Cap for comparison
    market_cap = stock.info.get('marketCap', 'Market Cap not available')

    return total_present_value, market_cap

# Main function
def main():
    ticker = input("Enter the stock ticker symbol: ")
    stock_data = fetch_stock_data(ticker)

    # Calculate 200-EMA
    stock_data = calculate_ema(stock_data)

    # Calculate the current price deviation from the 200-EMA
    current_deviation, current_close, current_ema = calculate_current_deviation(stock_data)

    # Get Sector and Market Cap
    try:
        sector, market_cap = get_sector_and_market_cap(ticker)
        formatted_market_cap = format_large_number(market_cap)
        print(f"Sector: {sector}")
        print(f"Market Capitalization: {formatted_market_cap}")
    except KeyError as e:
        print(f"Error retrieving sector or market capitalization: {e}")

    # Calculate ROE and Debt-to-Equity ratio
    try:
        roe, debt_to_equity_ratio = calculate_financial_ratios(ticker)
        print(f"Return on Equity (ROE): {roe:.2f}%")
        print(f"Debt-to-Equity Ratio: {debt_to_equity_ratio:.2f}")

        # Classify Debt-to-Equity Ratio
        de_classification = classify_de_ratio(debt_to_equity_ratio, sector)
        print(f"Debt-to-Equity Ratio Classification: {de_classification}")
    except KeyError as e:
        print(f"Error calculating financial ratios: {e}")

    # Calculate EPS and PE Ratio
    try:
        eps, pe_ratio = calculate_eps_pe(ticker)
        print(f"Earnings per Share (EPS): {eps:.2f}")
        print(f"Price-to-Earnings (PE) Ratio: {pe_ratio:.2f}")

        # Classify PE Ratio
        pe_classification = classify_pe_ratio(pe_ratio, sector)
        print(f"PE Ratio Classification: {pe_classification}")

        # Classify EPS
        eps_classification = classify_eps(eps, sector)
        print(f"EPS Classification: {eps_classification}")
    except KeyError as e:
        print(f"Error calculating EPS or PE ratio: {e}")

    # Perform DCF Analysis
    try:
        dcf_value, market_cap = perform_dcf_analysis(ticker, sector)
        print(f"DCF Valuation: {format_large_number(dcf_value)}")

        # Classify DCF Valuation
        dcf_classification = classify_dcf_value(dcf_value, market_cap, sector)
        print(f"DCF Valuation Classification: {dcf_classification}")
    except KeyError as e:
        print(f"Error performing DCF analysis: {e}")

    # Display the results
    print("\nCurrent Values for", ticker)
    print(f"Current Close Price: {current_close:.2f}")
    print(f"Current 200-EMA: {current_ema:.2f}")
    print(f"Current Price Deviation from 200-EMA: {current_deviation:.2f}%")

if __name__ == "__main__":
    main()

import yfinance as yf
import pandas as pd
import json

# Function to fetch stock data
def fetch_stock_data(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1y")  # Fetch data for the last year
    return hist

# Function to calculate the 200-day EMA
def calculate_ema(df, span=200):
    df['200-EMA'] = df['Close'].ewm(span=span, adjust=False).mean()
    return df

# Function to calculate the current price deviation from the 200-EMA
def calculate_current_deviation(df):
    current_close = df['Close'].iloc[-1]
    current_ema = df['200-EMA'].iloc[-1]
    current_deviation = ((current_close - current_ema) / current_ema) * 100
    return current_deviation, current_close, current_ema

# Function to calculate ROE, Debt-to-Equity ratio, and other financial metrics
def calculate_financial_ratios(ticker):
    stock = yf.Ticker(ticker)
    balance_sheet = stock.balance_sheet
    financials = stock.financials

    # Attempt to get Shareholders' Equity from the balance sheet
    equity_keys = ['Stockholders Equity', 'Common Stock Equity']
    for key in equity_keys:
        if key in balance_sheet.index:
            shareholders_equity = balance_sheet.loc[key].iloc[0]
            break
    else:
        raise KeyError("Shareholders' equity not found in balance sheet.")

    # Attempt to get Total Debt from the balance sheet
    total_debt_keys = ['Total Debt', 'Total Liabilities Net Minority Interest']
    for key in total_debt_keys:
        if key in balance_sheet.index:
            total_debt = balance_sheet.loc[key].iloc[0]
            break
    else:
        raise KeyError("Total Debt not found in balance sheet.")

    # Extracting the latest net income
    net_income = financials.loc['Net Income'].iloc[0]

    # Calculating ROE
    roe = (net_income / shareholders_equity) * 100

    # Calculating Debt-to-Equity Ratio
    debt_to_equity_ratio = total_debt / shareholders_equity

    return roe, debt_to_equity_ratio

# Function to calculate EPS and PE Ratio
def calculate_eps_pe(ticker):
    stock = yf.Ticker(ticker)
    # Earnings per share (EPS)
    eps = stock.info.get('trailingEps')
    if eps is None:
        raise KeyError("Earnings per Share (EPS) data not found.")

    # Current price
    current_price = stock.history(period="1d")['Close'].iloc[-1]

    # Calculate PE Ratio
    pe_ratio = current_price / eps
    return eps, pe_ratio

# Function to classify PE ratio based on sector-specific thresholds
def classify_pe_ratio(pe_ratio, sector):
    if pe_ratio < 0:
        return "Avoid"

    sector_pe_thresholds = {
        "Technology": (10, 20, 30, 40),
        "Healthcare": (8, 16, 24, 32),
        "Financial Services": (5, 10, 15, 20),
        "Consumer Cyclical": (10, 18, 25, 32),
        "Consumer Defensive": (10, 18, 22, 26),
        "Energy": (5, 10, 15, 20),
        "Utilities": (8, 14, 20, 25),
        "Basic Materials": (5, 10, 15, 20),
        "Industrials": (8, 14, 20, 26),
        "Real Estate": (10, 16, 22, 28)
        # Add other sectors as needed
    }

    thresholds = sector_pe_thresholds.get(sector, (10, 20, 30, 40))  # Default thresholds

    if pe_ratio < thresholds[0]:
        return "Very Low"
    elif thresholds[0] <= pe_ratio < thresholds[1]:
        return "Low"
    elif thresholds[1] <= pe_ratio < thresholds[2]:
        return "Medium"
    elif thresholds[2] <= pe_ratio < thresholds[3]:
        return "High"
    else:
        return "Very High"

# Function to classify Debt-to-Equity ratio based on sector-specific thresholds
def classify_de_ratio(de_ratio, sector):
    sector_de_thresholds = {
        "Technology": (0.1, 0.5, 1.0, 2.0),
        "Healthcare": (0.2, 0.6, 1.2, 2.5),
        "Financial Services": (1.0, 2.0, 3.0, 5.0),
        "Consumer Cyclical": (0.3, 0.7, 1.5, 3.0),
        "Consumer Defensive": (0.2, 0.6, 1.2, 2.0),
        "Energy": (0.3, 0.8, 1.5, 3.0),
        "Utilities": (0.5, 1.0, 2.0, 3.5),
        "Basic Materials": (0.2, 0.7, 1.5, 2.5),
        "Industrials": (0.3, 0.8, 1.5, 2.5),
        "Real Estate": (0.5, 1.0, 2.0, 3.0)
        # Add other sectors as needed
    }

    thresholds = sector_de_thresholds.get(sector, (0.5, 1.0, 2.0, 3.0))  # Default thresholds

    if de_ratio < thresholds[0]:
        return "Very Low"
    elif thresholds[0] <= de_ratio < thresholds[1]:
        return "Low"
    elif thresholds[1] <= de_ratio < thresholds[2]:
        return "Medium"
    elif thresholds[2] <= de_ratio < thresholds[3]:
        return "High"
    else:
        return "Very High"

# Function to classify EPS based on sector-specific thresholds
def classify_eps(eps, sector):
    sector_eps_thresholds = {
        "Technology": (1, 3, 5, 10),
        "Healthcare": (0.5, 2, 4, 8),
        "Financial Services": (0.5, 2, 4, 8),
        "Consumer Cyclical": (1, 3, 5, 10),
        "Consumer Defensive": (0.5, 2, 4, 8),
        "Energy": (0.5, 2, 4, 8),
        "Utilities": (0.5, 1, 2, 4),
        "Basic Materials": (0.5, 2, 4, 8),
        "Industrials": (0.5, 2, 4, 8),
        "Real Estate": (0.5, 2, 4, 8)
        # Add other sectors as needed
    }

    thresholds = sector_eps_thresholds.get(sector, (1, 3, 5, 10))  # Default thresholds

    if eps < thresholds[0]:
        return "Very Low"
    elif thresholds[0] <= eps < thresholds[1]:
        return "Low"
    elif thresholds[1] <= eps < thresholds[2]:
        return "Medium"
    elif thresholds[2] <= eps < thresholds[3]:
        return "High"
    else:
        return "Very High"

# Function to get the sector and market capitalization of the stock
def get_sector_and_market_cap(ticker):
    stock = yf.Ticker(ticker)
    sector = stock.info.get('sector', 'Sector not available')
    market_cap = stock.info.get('marketCap', 'Market Cap not available')
    return sector, market_cap

# Function to format large numbers in billions, millions, or trillions
def format_large_number(value):
    if value >= 1_000_000_000_000:
        return f"${value / 1_000_000_000_000:.2f} Trillion"
    elif value >= 1_000_000_000:
        return f"${value / 1_000_000_000:.2f} Billion"
    elif value >= 1_000_000:
        return f"${value / 1_000_000:.2f} Million"
    else:
        return f"${value:.2f}"

# Function to get sector-specific discount rates
def get_discount_rate_by_sector(sector):
    sector_discount_rates = {
        "Technology": 0.12,
        "Healthcare": 0.10,
        "Financial Services": 0.08,
        "Consumer Cyclical": 0.09,
        "Consumer Defensive": 0.07,
        "Energy": 0.13,
        "Utilities": 0.06,
        "Basic Materials": 0.11,
        "Industrials": 0.09,
        "Real Estate": 0.08
        # Add other sectors as needed
    }
    return sector_discount_rates.get(sector, 0.10)  # Default discount rate

# Function to perform DCF analysis
def perform_dcf_analysis(ticker, sector, growth_rate=0.06, projection_period=5):
    stock = yf.Ticker(ticker)

    # Estimate Free Cash Flow (FCF) for the last year
    fcf = stock.cashflow.loc['Free Cash Flow'].iloc[0]

    # Get the sector-specific discount rate
    discount_rate = get_discount_rate_by_sector(sector)

    # Calculate the present value of projected FCF
    present_value_of_fcfs = 0
    for year in range(1, projection_period + 1):
        fcf_future = fcf * ((1 + growth_rate) ** year)
        present_value_of_fcfs += fcf_future / ((1 + discount_rate) ** year)

    # Calculate the terminal value
    terminal_value = (fcf * ((1 + growth_rate) ** projection_period)) / (discount_rate - growth_rate)
    present_value_of_terminal_value = terminal_value / ((1 + discount_rate) ** projection_period)

    # Total present value
    total_present_value = present_value_of_fcfs + present_value_of_terminal_value

    # Market Cap for comparison
    market_cap = stock.info.get('marketCap', 0)

    return total_present_value, market_cap

# Function to classify DCF value based on sector-specific thresholds
def classify_dcf_value(dcf_value, market_cap, sector):
    sector_dcf_thresholds = {
        "Technology": (0.5, 1, 2, 3),
        "Healthcare": (0.5, 1, 2, 3),
        "Financial Services": (0.5, 1, 2, 3),
        "Consumer Cyclical": (0.5, 1, 2, 3),
        "Consumer Defensive": (0.5, 1, 2, 3),
        "Energy": (0.5, 1, 2, 3),
        "Utilities": (0.5, 1, 2, 3),
        "Basic Materials": (0.5, 1, 2, 3),
        "Industrials": (0.5, 1, 2, 3),
        "Real Estate": (0.5, 1, 2, 3)
        # Add other sectors as needed
    }

    thresholds = sector_dcf_thresholds.get(sector, (0.5, 1, 2, 3))  # Default thresholds

    market_cap = market_cap or 0  # Ensure market_cap is not None

    ratio = dcf_value / market_cap if market_cap > 0 else 0

    if ratio < thresholds[0]:
        return "Very Low"
    elif thresholds[0] <= ratio < thresholds[1]:
        return "Low"
    elif thresholds[1] <= ratio < thresholds[2]:
        return "Medium"
    elif thresholds[2] <= ratio < thresholds[3]:
        return "High"
    else:
        return "Very High"

# Main function
def main():
    ticker = input("Enter the stock ticker symbol: ")
    stock_data = fetch_stock_data(ticker)

    # Calculate 200-EMA
    stock_data = calculate_ema(stock_data)

    # Calculate the current price deviation from the 200-EMA
    current_deviation, current_close, current_ema = calculate_current_deviation(stock_data)

    # Get Sector and Market Cap
    try:
        sector, market_cap = get_sector_and_market_cap(ticker)
        formatted_market_cap = format_large_number(market_cap)
    except KeyError as e:
        sector = 'Sector not available'
        formatted_market_cap = 'Market Cap not available'

    # Calculate ROE and Debt-to-Equity ratio
    try:
        roe, debt_to_equity_ratio = calculate_financial_ratios(ticker)
        de_classification = classify_de_ratio(debt_to_equity_ratio, sector)
    except KeyError as e:
        roe = None
        debt_to_equity_ratio = None
        de_classification = 'Error'

    # Calculate EPS and PE Ratio
    try:
        eps, pe_ratio = calculate_eps_pe(ticker)
        pe_classification = classify_pe_ratio(pe_ratio, sector)
        eps_classification = classify_eps(eps, sector)
    except KeyError as e:
        eps = None
        pe_ratio = None
        pe_classification = 'Error'
        eps_classification = 'Error'

    # Perform DCF Analysis
    try:
        dcf_value, market_cap = perform_dcf_analysis(ticker, sector)
        dcf_classification = classify_dcf_value(dcf_value, market_cap, sector)
    except KeyError as e:
        dcf_value = None
        dcf_classification = 'Error'

    # Create a result dictionary
    result = {
        "ticker": ticker,
        "current_values": {
            "current_close_price": current_close,
            "current_200_ema": current_ema,
            "current_price_deviation_200_ema": current_deviation
        },
        "sector": sector,
        "market_cap": formatted_market_cap,
        "roe": roe,
        "debt_to_equity_ratio": debt_to_equity_ratio,
        "debt_to_equity_ratio_classification": de_classification,
        "eps": eps,
        "pe_ratio": pe_ratio,
        "pe_ratio_classification": pe_classification,
        "eps_classification": eps_classification,
        "dcf_value": format_large_number(dcf_value) if dcf_value else None,
        "dcf_classification": dcf_classification
    }

    # Print result as JSON
    print(json.dumps(result, indent=4))

if __name__ == "__main__":
    main()

Sector: Technology
Market Capitalization: $3.48 Trillion
Return on Equity (ROE): 156.08%
Debt-to-Equity Ratio: 1.99
Debt-to-Equity Ratio Classification: High
Earnings per Share (EPS): 6.56
Price-to-Earnings (PE) Ratio: 34.84
PE Ratio Classification: High
EPS Classification: High
DCF Valuation: $1.68 Trillion
DCF Valuation Classification: Very Low

Current Values for AAPL
Current Close Price: 228.57
Current 200-EMA: 204.70
Current Price Deviation from 200-EMA: 11.66%
